<a href="https://colab.research.google.com/github/akaul09/LanguageTranslationModel/blob/main/Project1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, BertModel
from torch.utils.data import DataLoader, TensorDataset
import torch
from torchtext.data.utils import get_tokenizer


In [4]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertModel.from_pretrained("bert-base-multilingual-cased")

In [20]:
df = pd.read_csv("eng_-french.csv")
english = df["English words/sentences"]
french = df["French words/sentences"]

In [21]:
input_ids = []
attention_masks = []

for english_phrase, french_translation in zip(english, french):
    encoded = tokenizer(english_phrase, french_translation, return_tensors='pt', padding=True, truncation=True)
    input_ids.append(encoded['input_ids'])
    attention_masks.append(encoded['attention_mask'])